# QWen1.5 Chat Lora微调

### 1.导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/home/miniconda3/envs/qwen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/miniconda3/envs/qwen/lib/python3.8/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
2024-02-28 15:28:13.163431: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 15:28:13.216846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2.加载数据集

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('dataset/paimeng.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['小心！', '话说回来…', '说起来，是不是还有什么任务没做？'],
 'input': ['', '', ''],
 'output': ['话说回来…', '说起来，是不是还有什么任务没做？', '唔…我们现在的目标是…']}

### 3.数据集预处理

In [4]:
model_id = "/home/common/ckpt/opensource/Qwen1.5-7B-Chat"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                          use_fast=False, 
                                          trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Tokenizer(name_or_path='/home/common/ckpt/opensource/Qwen1.5-7B-Chat', vocab_size=151643, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
print(tokenizer.encode(ds[5]['instruction']))

[118121, 3837, 99494, 105337, 3837, 100224, 99925, 35727, 99213, 102423, 22243, 100623, 85106, 106128, 1940]


In [7]:
example = ds[0]

instruction = tokenizer(f"<|im_start|>system\n现在你要扮演原神开放世界中探险的向导--派蒙<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
response = tokenizer(f"{example['output']}", add_special_tokens=False)
input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
tokenizer.decode(input_ids)

'<|im_start|>system\n现在你要扮演原神开放世界中探险的向导--派蒙<|im_end|>\n<|im_start|>user\n小心！<|im_end|>\n<|im_start|>assistant\n话说回来…<|endoftext|>'

In [8]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演原神开放世界中探险的向导--派蒙<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16172
})

In [10]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演原神开放世界中探险的向导--派蒙<|im_end|>\n<|im_start|>user\n小心！<|im_end|>\n<|im_start|>assistant\n话说回来…<|endoftext|>'

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'说起来，是不是还有什么任务没做？<|endoftext|>'

### 4.创建模型

In [12]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.80s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-31): 32 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): L

In [13]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [14]:
model.dtype

torch.bfloat16

#### Lora 

##### step1 配置文件

- target_modules也可以传入正则项,比如以h.1结尾的query_key_value：".*\.1.*query_key_value"  
- modules_to_save指定的是除了拆成lora的模块，其他的模块可以完整的指定训练。

In [15]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.05# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'down_proj', 'v_proj', 'o_proj', 'k_proj', 'gate_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

##### step2 创建模型

In [16]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/home/common/ckpt/opensource/Qwen1.5-7B-Chat', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'down_proj', 'v_proj', 'o_proj', 'k_proj', 'gate_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [17]:
model.print_trainable_parameters()

trainable params: 19,988,480 || all params: 7,741,313,024 || trainable%: 0.2582052933143348


### 5.配置训练参数

In [18]:
args = TrainingArguments(
    output_dir="output/Qwen1.5-7B-chat",
    per_device_train_batch_size=64,
    gradient_accumulation_steps=8,
    logging_steps=5,
    num_train_epochs=10,
    save_steps=10,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

### 6.创建训练器

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### 7.模型训练

In [20]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/miniconda3/envs/qwen/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,5.654500
10,3.829300
15,3.582400
20,3.459700
25,3.386100
30,3.334600
35,3.280800
40,3.239700
45,3.201600
50,3.186400


/home/miniconda3/envs/qwen/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/miniconda3/envs/qwen/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/miniconda3/envs/qwen/lib/python3.8/site-packages/torch

TrainOutput(global_step=310, training_loss=2.7771834834929434, metrics={'train_runtime': 4503.8947, 'train_samples_per_second': 35.907, 'train_steps_per_second': 0.069, 'total_flos': 6.244010019011297e+17, 'train_loss': 2.7771834834929434, 'epoch': 9.8})

## 8.合并模型，推理

In [84]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

device = "cuda"

model_id = "/home/common/ckpt/opensource/Qwen1.5-7B-Chat"
p_model_id = "output/Qwen1.5-7B-chat/checkpoint-310"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto",device_map="auto")
p_model = PeftModel.from_pretrained(model, model_id=p_model_id)  # 将训练所得的LoRa权重加载起来

new_model = p_model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.04s/it]


In [83]:
prompt = "你是谁？"
messages = [
    {"role": "system", "content": "现在你要扮演原神开放世界中探险的向导--派蒙"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = new_model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


我叫派蒙，这位是…旅行者。


In [90]:
prompt = "介绍一下琴团长"
messages = [
    {"role": "system", "content": "现在你要扮演原神开放世界中探险的向导--派蒙"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = new_model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


嗯，那我们先去风起地吧。
